# Required Packages

In [1]:
%pip install langchain
%pip install pinecone-client
%pip install pypdf
%pip install openai
%pip install tiktoken

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


# Imports

In [2]:
from langchain.document_loaders import PyPDFDirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings
from langchain.llms import OpenAI
from langchain.vectorstores import Pinecone
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate
import os

Load and split into chunks the pdf data

In [3]:
loader=PyPDFDirectoryLoader("pdfs")

In [4]:
data=loader.load()

In [5]:
data[0]

Document(page_content='Orca-Math: Unlocking the potential of\nSLMs in Grade School Math\nArindam Mitra∗, Hamed Khanpour, Corby Rosset, Ahmed Awadallah\nMicrosoft Research\nAbstract\nWe show that an SLM can reach ∼87%pass@1 on GSM8K while trained on only 200K\nsynthetic math problems. Mathematical word problem-solving has long been recognized as\na complex task for small language models (SLMs). A recent study hypothesized that the\nsmallest model size, needed to achieve over 80% accuracy on the GSM8K benchmark, is\n34 billion parameters. To reach this level of performance with smaller models, researcher\noften train SLMs to generate Python code or use tools to help avoid calculation errors.\nAdditionally, they employ ensembling, where outputs of up to 100 model runs are combined\nto arrive at a more accurate result. Result selection is done using consensus, majority vote\nor a separate a verifier model used in conjunction with the SLM. Ensembling provides a\nsubstantial boost in accurac

In [6]:
text_splitter=RecursiveCharacterTextSplitter(chunk_size=500,chunk_overlap=20)

In [7]:
text_chunks=text_splitter.split_documents(data)

In [8]:
print(text_chunks[0].page_content)

Orca-Math: Unlocking the potential of
SLMs in Grade School Math
Arindam Mitra∗, Hamed Khanpour, Corby Rosset, Ahmed Awadallah
Microsoft Research
Abstract
We show that an SLM can reach ∼87%pass@1 on GSM8K while trained on only 200K
synthetic math problems. Mathematical word problem-solving has long been recognized as
a complex task for small language models (SLMs). A recent study hypothesized that the
smallest model size, needed to achieve over 80% accuracy on the GSM8K benchmark, is


In [9]:
len(text_chunks)

93

In [10]:
# last chunk
print(text_chunks[92].page_content)

Chen. Mammoth: Building math generalist models through hybrid instruction tuning. arXiv
preprint arXiv:2309.05653 , 2023.
[44]Yifan Zhang, Yifan Luo, Yang Yuan, and Andrew Chi-Chih Yao. Templatemath: Syntactic
data generation for mathematical problems, 2024.
[45]Chunting Zhou, Pengfei Liu, Puxin Xu, Srini Iyer, Jiao Sun, Yuning Mao, Xuezhe Ma, Avia
Efrat, PingYu, LiliYu, etal. Lima: Lessismoreforalignment. arXiv preprint arXiv:2305.11206 ,
2023.
14


In [11]:
key=os.getenv("OPENAI_API_KEY")

In [12]:
embedding=OpenAIEmbeddings()

C:\Users\victo\AppData\Roaming\Python\Python39\site-packages\langchain_core\_api\deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.embeddings.openai.OpenAIEmbeddings` was deprecated in langchain-community 0.0.9 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAIEmbeddings`.
  warn_deprecated(


Embedding example

In [13]:
embedding.embed_query("Hi there what is up?")

[-0.01029803499474263,
 0.0032209641293987805,
 -0.008462101294422243,
 -0.032296915057944915,
 -0.01480382996456017,
 0.020156480190901962,
 -0.010744089102660904,
 0.005879512391898481,
 -0.009548147172210522,
 -0.008306952282447837,
 0.024733384426049245,
 -0.0032274286715643806,
 -0.02205705977853959,
 0.0065776838771057855,
 0.02318189244366023,
 -0.007841504315202141,
 0.03304680101783206,
 -0.01729914743143302,
 0.02539277005524667,
 -0.019018718907111362,
 -0.024927321156678492,
 -0.0012242250300205713,
 -0.0010698837074671083,
 0.009599863509535324,
 -0.0005151441496956622,
 0.005258914947017138,
 0.016006237135668015,
 -0.010446719697382055,
 0.012735171814955697,
 -0.039304490872647806,
 0.001974921473296101,
 0.012883856517595122,
 -0.019729820873633588,
 -0.009063305345637405,
 -0.0055756782116234095,
 -0.017467225062077384,
 -0.00391428784826609,
 -0.009386532919578656,
 0.024035212940841943,
 -0.008500889013077084,
 0.0066584910034217185,
 -0.01652340050891603,
 0.002113

In [14]:
PINECONE_API_KEY=os.getenv("PINECONE_API_KEY")
PINECONE_API_ENV=os.getenv("PINECONE_API_ENV")